In [ ]:
import digitalhub as dh

project = dh.get_or_create_project("demo-early-exit-eng")

In [ ]:
project.log_model(
    name="early-exit-model",
    kind="model",
    source="/home/user/trained_model/mod032-transformer",
    algorithm="early-exit",
    framework="pythorch"
)

In [ ]:
project.log_artifact(
    name="lexicon.lex",
    kind="artifact",
    source="/home/user/upload/lexicon.lex"
)

In [ ]:
project.log_artifact(
    name="bpe-256.model",
    kind="artifact",
    source="/home/user/upload/bpe-256.model"
)

In [ ]:
project.log_artifact(
    name="bpe-256.lex",
    kind="artifact",
    source="/home/user/upload/bpe-256.lex"
)

In [ ]:
project.log_artifact(
    name="bpe-256.tok",
    kind="artifact",
    source="/home/user/upload/bpe-256.tok"
)

In [ ]:
func = project.new_function(name="serve_function",
                            kind="python",
                            python_version="PYTHON3_10",
                            code_src="git+https://github.com/tn-aixpa/audio-early-exit-transformer",
                            handler="serve_model:serve_multipart",
                            init_function="init",
                            requirements=["torch==2.5.0", "torchaudio==2.5.0", "tensorboard==2.18.0",
                                "flashlight==0.1.1", "flashlight-text==0.0.7", "sentencepiece==0.2.0",
                                "soundfile==0.12.1", "editdistance==0.8.1", "multipart==1.2.1"])

In [ ]:
run = func.run(
    action="serve",
    profile="1xa100",
    resources = {"mem":{"requests": "1Gi",}},
    init_parameters = {
        "model_name":"early-exit-model",
        "lexicon":"lexicon.lex",
        "sp_model":"bpe-256.model",
        "sp_lexicon":"bpe-256.lex",
        "sp_tokens":"bpe-256.tok"
    },    
    volumes=[
        {
            "volume_type": "persistent_volume_claim",
            "name": "early-exit-demo-serve",
            "mount_path": "/data",
            "spec": { "size": "5Gi" }        
        }
    ]
)

In [ ]:
project.log_artifact(
    name="test-audio-eng.wav",
    kind="artifact",
    source="/home/user/upload/test-audio-eng.wav"
)

In [ ]:
run.refresh()
file_path = "/home/user/upload/test-audio-eng.wav"
with open(file_path, "rb") as file:
    files = {'file': file}
    response = run.invoke(method="POST", files=files)
    print(response.status_code)
    print(response.text)

In [ ]:
#run.refresh()
#json = {"name": "test-audio-eng.wav"}
#response = run.invoke(method="POST", json=json)
#print(response.json())

In [ ]:
#function = project.new_function(
#    kind='container',
#    name='serve_eng_function',
#    image="ghcr.io/tn-aixpa/audio-early-exit-eng:0.1",
#    args=["early-exit-eng-model"]
#)

In [ ]:
#run = function.run(
#    action="serve",
#    profile="1xa100",
#    fs_group=100,
#    service_ports = [
#        {
#            "port": 8051,
#            "target_port": 8051
#        }
#    ],
#    volumes=[
#        {
#            "volume_type": "persistent_volume_claim",
#            "name": "early-exit-demo-serve",
#            "mount_path": "/data",
#            "spec": {
#                "claim_name": "early-exit-demo-serve"
#            }
#        }
#    ]
#)